In [ ]:
import requests
import json
import pandas as pd
import base64

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
url = ""
clientID = ""
tgt = ""
token = ""

In [ ]:
def getToken(url, clientID, tgt):
    url = url + "auth/login?client_id=" + clientID + "&tgt=" + tgt
    payload = {}
    headers = {}   
    return json.loads(requests.request("POST", url, headers=headers, data=payload, verify=False).text)["token"]

In [ ]:
def getAchievements(url, token, taskID):
    url = url + "task_run/" + taskID + "/achievements"
    print(url)
    payload = {}
    headers = {
        'Authorization':
            token
    }
    return json.loads(requests.request("GET", url, headers=headers, data=payload, verify=False).text)

In [ ]:
def getVulns(url, token, taskID):
    url = url + "task_run/" + taskID + "/vulnerabilities"
    print(url)
    payload = {}
    headers = {
        'Authorization':
            token
    }
    return json.loads(requests.request("GET", url, headers=headers, data=payload, verify=False).text)

In [ ]:
def getHosts(url, token, taskID):
    url = url + "task_run/" + taskID + "/hosts"
    print(url)
    payload = {}
    headers = {
        'Authorization':
            token
    }
    return json.loads(requests.request("GET", url, headers=headers, data=payload, verify=False).text)

In [ ]:
taskID = ""
token = getToken(url, clientID, tgt)
df_achvs = pd.DataFrame(getAchievements(url, token, taskID)["achievements"])
df_vulns = pd.DataFrame(getVulns(url, token, taskID)["vulnerabilities"])
df_hosts = pd.DataFrame(getHosts(url, token, taskID)["hosts"])

In [ ]:
# df_achvs
df_vulns = df_vulns[df_vulns.id != "pcysys_data_analysis_9"].copy()
# df_hosts

In [ ]:
# Creates the header used for API authentication with ZenGRC
# Output: Authorization Header
def formatZenAuthHeader():
    zenkey = ""
    message_bytes = zenkey.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode("ascii")
    header = "Basic " + base64_message
    
    return header

In [ ]:
# Gets the vulnerability custom attributes in Zen
# Output: All vulnerabilities populated in ZenGRC
def getZenVulnerabilities():
    payload = {}
    headers = {
        'Authorization':
            formatZenAuthHeader()
    }
    
    url = "https://psbrands.api.zengrc.com/api/v2/vulnerabilities"
    return json.loads(requests.request("GET", url, headers=headers, data=payload, verify=False).text)["data"]

In [ ]:
# Gets the vulnerability custom attributes in Zen
# Output: Custom attribute data in JSON format
def getZenVulnCustomAttributes():
    payload = {}
    headers = {
        'Authorization':
            formatZenAuthHeader()
    }
    
    url = "https://psbrands.api.zengrc.com/api/v2/vulnerabilities/custom_attributes"
    return json.loads(requests.request("GET", url, headers=headers, data=payload, verify=False).text)

In [ ]:
# Formats Vulnerability for ZenGRC Post Vuln
# Input: Series (or row) from Dataframe containing all vuln data from PenTest
# Output: Data in JSON format
def formatZenVuln(vulnData):
    base = {
        "data": 
        [
            {
                "code": vulnData["id"],
                "contact": 12,
                "custom_attributes": {
                    "43": "nothing",
                    "44": vulnData["port"],
                    "46": "nothing",
                    "47": vulnData["found_on"],
                    "48": str(vulnData["severity"]),
                    "59": vulnData["summary"],
                    "60": vulnData["insight"],
                    "61": vulnData["remediation"],
                },
                "description": "description",
                "end_date": "2017-12-01",
                "owners": [5],
                "contact": 5,
                "status": "Identified",
                "title": vulnData["name"],
                "url": "http://www.primesourcebp.com"
            }
        ]
    }    
    
    return base

In [ ]:
# Checks if Vulnerability already exists in ZenGRC
# Input: Vulnerability ID
# Output: True/False based on existence
def checkZenOverlap(vulnID):
    vulnsList = []
    for i in range(0, len(getZenVulnerabilities())): vulnsList.append(getZenVulnerabilities()[i]["code"])
    for i in vulnsList:
        if (i == vulnID):
            return True
    return False

In [ ]:
# Posts a Pentera vulnerability to ZenGRC
# Input: JSON Formatted Vulnerability Data
# Output: Post Response?
def postZenVuln(vulnData):
    url = "https://psbrands.api.zengrc.com/api/v2/vulnerabilities"
    headers = {
        'Authorization':
            formatZenAuthHeader(),
        'Content-Type': 'application/json'
    }
    
    for index, row in vulnData.iterrows():
        vuln = formatZenVuln(row)
        if checkZenOverlap(vuln["data"][0]["code"]) == True:
            print("Vulnerability Already Exists... Append!")
            updateZenVuln(vuln)
        else:
            print(json.loads(requests.request("POST", url, headers=headers, data=json.dumps(vuln), verify=False).text))

In [ ]:
def formatZenDataFromResponseVuln(responseData):
    responseDataID = responseData["id"]
    code = responseData["code"]
    port = responseData["custom_attributes"]["44"]["value"]
    found_on = responseData["custom_attributes"]["47"]["value"]
    severity = responseData["custom_attributes"]["48"]["value"]
    summary = responseData["custom_attributes"]["59"]["value"]
    insight = responseData["custom_attributes"]["60"]["value"]
    remediation = responseData["custom_attributes"]["61"]["value"]
    name = responseData["title"]
    
    base = {
        "data": 
        [
            {
                "id": responseDataID,
                "code": code,
                "contact": 12,
                "custom_attributes": {
                    "43": "nothing",
                    "44": port,
                    "46": "nothing",
                    "47": str(found_on),
                    "48": severity,
                    "59": str(summary),
                    "60": str(insight),
                    "61": str(remediation),
                },
                "description": "description",
                "end_date": "2017-12-01",
                "owners": [5],
                "contact": 5,
                "status": "Identified",
                "title": name,
                "url": "http://www.primesourcebp.com"
            }
        ]
    }
    return base

In [ ]:
# Update an existing Vulnerability entry
# Input: Single JSON Formatted Vulnerability
# Output: Put response
def updateZenVuln(vuln):
    vulnID = vuln["data"][0]["code"]
    vulnsList = []
    existingVuln = ""
    
    for i in range(0, len(getZenVulnerabilities())): 
        vulnsList.append(getZenVulnerabilities()[i])
        
    for i in vulnsList:
        if (i["code"] == vulnID):
            existingVuln = i
    
    formattedExistingVuln = formatZenDataFromResponseVuln(existingVuln)
    
    #     if (vuln["data"][0]["custom_attributes"]["47"].split()[1] ==  formattedExistingVuln["data"][0]["custom_attributes"]["47"].split()[1]):
    #         print("Duplicate Value")
    #         return
    
    #     else:
    formattedExistingVuln["data"][0]["custom_attributes"]["47"] = formattedExistingVuln["data"][0]["custom_attributes"]["47"] + " " + vuln["data"][0]["custom_attributes"]["47"]   
    
    payload = json.dumps(formattedExistingVuln)  
    
    url = "https://psbrands.api.zengrc.com/api/v2/vulnerabilities"
    headers = {
        'Authorization':
            formatZenAuthHeader(),
        'Content-Type': 'application/json'
    }
    
    return json.loads(requests.request("PUT", url, headers=headers, data=payload).text)
    

In [ ]:
df_vulns["port"] = df_vulns["port"].fillna(0)
df_vulns["found_on"] = df_vulns["found_on"].fillna("N/A")
df_vulns["summary"] = df_vulns["summary"].fillna("N/A")
df_vulns["severity"] = df_vulns["severity"].fillna("N/A")
df_vulns["insight"] = df_vulns["insight"].fillna("N/A")
postZenVuln(df_vulns)